#### Middleware
Middleware provides a way to more tightly control what happens inside the agent. Middleware is useful for the following:

- Tracking agent behaviour with logging, analytics and debugging.
- Transforming prompts, tool selection and output formatting.
- Applying retries, fallbacks and early termination logic.
- Applying rate limits, guardrails and PII detection.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

#### Summarization MiddleWare
Automatically summarize conversation history when approaching token limits, preserving recent messages when compressing older context. Summarization is useful for the following:
- Long running conversations that exceed context windows.
- Multi-turn dialogues with extensive history
- Applications when preservinf full conversation context matters.

In [4]:
from langchain.agents import create_agent 
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import HumanMessage, SystemMessage

### Message based summarization
agent=create_agent(
    model="groq:llama-3.3-70b-versatile",
    checkpointer=InMemorySaver(), #saving conversation history locally
    middleware=[
        SummarizationMiddleware(model="groq:llama-3.3-70b-versatile",
        trigger=("messages",10), #trigger when i get 10 messages
        keep=("messages",4), )#summarize the context and keep the recent top 4 messages

    ]
)

In [9]:
### Run with a thread id 
config={"configurable":{"thread_id":"test-1"}}

In [10]:
### Alternative test data
questions=[
    "what is 2+2",
    "What is 10*5?",
    "what is 100/4?",
    "what is 15-7?",
    "What is 3*3?",
    "What is 4*4?",]

In [11]:
for q in questions:
    response=agent.invoke({"messages":[HumanMessage(content=q)]},config)
    print(f"Messages:{response}")
    print(f"Messages:{len(response['messages'])}")

Messages:{'messages': [HumanMessage(content='what is 2+2', additional_kwargs={}, response_metadata={}, id='a0d8c4fd-184c-4b7f-afa5-b60e1bf07006'), AIMessage(content='2 + 2 = 4', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 41, 'total_tokens': 49, 'completion_time': 0.011511762, 'completion_tokens_details': None, 'prompt_time': 0.00118493, 'prompt_tokens_details': None, 'queue_time': 0.048619216, 'total_time': 0.012696692}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c06d5113ec', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c38dd-1372-7631-a0be-af48d8995528-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 41, 'output_tokens': 8, 'total_tokens': 49})]}
Messages:2
Messages:{'messages': [HumanMessage(content='what is 2+2', additional_kwargs={}, response_metadata={}, id='a0d8c4fd-184c-4b7f-afa5-b60e1bf07006'), AIMessage(con

##### Based on token size


In [21]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import InMemorySaver

@tool
def search_hotels(city: str)->str:
    """Search hotels - returns a long response to use more tokens."""
    return f"""Hotels in {city}:
    1. Grand Hotel - 5 star, $350/night, spa, pool, gym
    2. City Inn - 4 star, $180/night, business center
    3. Budget Stay - 3 star, $75/night, free wifi"""

In [22]:
agent=create_agent(
    model="groq:llama-3.3-70b-versatile",
    tools=[search_hotels],
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            model="groq:llama-3.3-70b-versatile",
            trigger=("tokens",550),
            keep=("tokens",200)
        )
    ]
)
config={"configurable":{"thread_id":"test-2"}}

#Token counter (approximate)
def count_tokens(messages):
    total_chars=sum(len(str(m.content)) for m in messages)
    return total_chars//4  # 4 chars=1 token

In [23]:
#Run test
cities=["Paris","London","Tokyo","New York","Dubai","Singapore"]
for city in cities:
    response=agent.invoke(
        {"messages":[HumanMessage(content=f"Find hotels in {city}")]},
        config=config
    )
    tokens=count_tokens(response["messages"])
    print(f"{city}: ~{tokens} token, {len(response['messages'])} messages")
    print(f"{(response['messages'])}")

Paris: ~140 token, 6 messages
[HumanMessage(content='Find hotels in Paris', additional_kwargs={}, response_metadata={}, id='2dad49bb-175f-455c-9a23-d4ca57924031'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'bvyx5974g', 'function': {'arguments': '{"city":"Paris"}', 'name': 'search_hotels'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 228, 'total_tokens': 243, 'completion_time': 0.033966523, 'completion_tokens_details': None, 'prompt_time': 0.011531346, 'prompt_tokens_details': None, 'queue_time': 0.049114714, 'total_time': 0.045497869}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c06d5113ec', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c38ef-6558-71e0-8222-b65dc552d9db-0', tool_calls=[{'name': 'search_hotels', 'args': {'city': 'Paris'}, 'id': 'bvyx5974g', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadat

##### Based on fraction

In [ ]:
#just change the trigger and keep
#instead of tokens write fractions
#and give fraction values eg. 0.005=0.5% 

### Human In the Loop MiddleWare
Pause agent execution for human approval, editing, or rejection of tool calls before they execute. Human-in-the-loop is useful for the following:
- High-stakes operations requiring human approval (eg. database writes, financial transactions).
- Compliance workflows where human oversight is mandatory.
- Long-running conversations where human feedback guides the agent.

In [24]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import MemorySaver

def read_email_tool(email_id:str)->str:
    """Mock function to read an email by its ID."""
    return f"Email content for ID:{email_id}"

def send_email_tool(recipient: str, subject:str,body:str):
    """Mock function to send an email."""
    return f"Email send to {recipient} with subject '{subject}'"

In [25]:
agent=create_agent(
    model="groq:llama-3.3-70b-versatile",
    tools=[read_email_tool,send_email_tool],
    checkpointer=MemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "send_email_tool":{ #interrupt when send email tool is called
                    "allowed_decisions":["approve","edit","reject"] #human is allowed to say either of these things
                },
                "read_email_tool":False, #no need for human approval here
            }
        )
    ]
)

In [27]:
config={"configurable":{"thread_id":"test-approve"}}
#Step-1: Request
result=agent.invoke(
    {"messages":[HumanMessage(content="Send email to john@test.com with subject 'Hello' and body 'How are you?'")]},
    config=config
)

In [28]:
result

{'messages': [HumanMessage(content="Send email to john@test.com with subject 'Hello' and body 'How are you?'", additional_kwargs={}, response_metadata={}, id='7de059db-f438-40ad-9296-c7ed5c406b9f'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '0yzmdfgdg', 'function': {'arguments': '{"body":"How are you?","recipient":"john@test.com","subject":"Hello"}', 'name': 'send_email_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 309, 'total_tokens': 341, 'completion_time': 0.043852299, 'completion_tokens_details': None, 'prompt_time': 0.015727686, 'prompt_tokens_details': None, 'queue_time': 0.049058804, 'total_time': 0.059579985}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_dae98b5ecb', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c3902-ce59-7d53-8353-b61983da8fdf-0', tool_calls=[{'name': 'send_email_tool', 'args': {'b

In [30]:
#Step 2: Approve
from langgraph.types import Command
if "__interrupt__" in result:
    print("Paused! Approving...")

    result=agent.invoke(
        Command(
            resume={
                "decisions":[
            {"type":"approve"}]}
        ),
        config=config
    )
    print(f"Result:{result['messages'][-1].content}")

Paused! Approving...
Result:Email sent to john@test.com with subject 'Hello' and body 'How are you?'


In [31]:
### Reject
config={"configurable":{"thread_id":"test-reject"}}
result=agent.invoke(
    {"messages":[HumanMessage(content="Send email to john@test.com with subject 'Hello' and body 'How are you?'")]},
    config=config
)

In [32]:
#Step 2: Approve
from langgraph.types import Command
if "__interrupt__" in result:
    print("Paused! Approving...")

    result=agent.invoke(
        Command(
            resume={
                "decisions":[
            {"type":"reject"}]}
        ),
        config=config
    )
    print(f"Result:{result['messages'][-1].content}")

Paused! Approving...
Result:


In [33]:
### Edit
config={"configurable":{"thread_id":"test-edit"}}
result=agent.invoke(
    {"messages":[HumanMessage(content="Send email to john@test.com with subject 'Hello' and body 'How are you?'")]},
    config=config
)

In [36]:
result

{'messages': [HumanMessage(content="Send email to john@test.com with subject 'Hello' and body 'How are you?'", additional_kwargs={}, response_metadata={}, id='c08adf69-8333-49b0-916d-7f3e1426b9f2'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'd2ckfp7gb', 'function': {'arguments': '{"body":"How are you?","recipient":"john@test.com","subject":"Hello"}', 'name': 'send_email_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 309, 'total_tokens': 341, 'completion_time': 0.047792034, 'completion_tokens_details': None, 'prompt_time': 0.015631318, 'prompt_tokens_details': None, 'queue_time': 0.048685202, 'total_time': 0.063423352}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c06d5113ec', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c3909-f5ab-7c52-89fa-466fc49ce61b-0', tool_calls=[{'name': 'send_email_tool', 'args': {'b

In [37]:
if "__interrupt__" in result:
    print("Paused! Editing")
    result=agent.invoke(
        Command(
            resume={
                "decisions":[
                    {
                        "type":"edit",
                        "edited_action":{
                            "name":"send_email_tool",
                            "args":{
                                "recipient":"correct@gmail.com",
                                "subject":"Corrected Subject",
                                "body":"This was edited by human before sending"
                            }
                        }
                    }
                ]
            }
        ),
        config=config
    )

Paused! Editing
